# NBA Shot Selection Evolution (2000–2025)
This notebook uses `nba_api`, `pandas`, and `altair` to build interactive visualizations for an article-style narrative about how NBA shot selection and scoring geometry have changed from ~2000 to the present.

**Visualizations in this notebook:**

1. League-wide shot distribution timeline (stacked area, 2000–2025)
2. Event-driven turning points (annotated timeline)
3. Player-level shot selection evolution (interactive player dropdown)
4. Interactive / time-animated shot-chart heatmaps for selected players

> ⚠️ **Note:** This notebook is a template. You must run it in an environment with internet access and valid `nba_api` configuration. Some API calls may take time or require rate limiting.


In [ ]:
# If needed, install dependencies (uncomment as appropriate)
# !pip install nba_api altair pandas

import time
import pandas as pd
import altair as alt

# Altair settings
alt.data_transformers.disable_max_rows()
alt.renderers.enable('default')

# nba_api imports
from nba_api.stats.endpoints import LeagueDashTeamShotLocations, PlayerDashboardByShootingSplits, ShotChartDetail, PlayerCareerStats
from nba_api.stats.static import players as static_players


In [ ]:
### Helper functions

def season_str(start_year: int) -> str:
    # Return NBA season string like '2000-01' from starting year int.
    return f"{start_year}-{str(start_year + 1)[-2:]}"

def get_season_list(start=2000, end=2024):
    # Return list of season strings from start to end inclusive (e.g., 2000–2024 -> 2000-01..2024-25).
    return [season_str(y) for y in range(start, end + 1)]

def find_player_by_name(name: str):
    # Use nba_api static players to resolve a player's ID by full or partial name.
    all_players = static_players.get_players()
    matches = [p for p in all_players if name.lower() in p['full_name'].lower()]
    return matches  # list of dicts with 'id', 'full_name', etc.

SEASONS = get_season_list(2000, 2024)  # adjust end year as needed
SEASONS[:5], SEASONS[-5:]


## 1. League-wide shot distribution timeline

We aggregate `LeagueDashTeamShotLocations` over all teams for each season, and compute the share of field-goal attempts by shot zone. We'll use the `By Zone` distance range, which returns standard NBA shot zones (e.g., Restricted Area, Mid-Range, Corner 3).


In [ ]:
# Fetch league-wide shot distribution by season
# This may take a while; consider caching results to CSV once fetched.

zone_frames = []

for season in SEASONS:
    print(f'Fetching LeagueDashTeamShotLocations for {season}...')
    try:
        resp = LeagueDashTeamShotLocations(
            season=season,
            season_type_all_star='Regular Season',
            per_mode_simple='PerGame',
            distance_range='By Zone'
        )
        df = resp.get_data_frames()[0]
        df['SEASON'] = season
        zone_frames.append(df)
        time.sleep(1.0)  # basic rate limiting
    except Exception as e:
        print(f'Failed for {season}: {e}')

df_league_raw = pd.concat(zone_frames, ignore_index=True)
df_league_raw.head()


In [ ]:
# Transform league shot-location data into long format for Altair

# The LeagueDashTeamShotLocations output typically has columns like:
# 'SHOT_ZONE_BASIC', 'FGM', 'FGA', etc. We'll aggregate FGA by SEASON + SHOT_ZONE_BASIC.

df_league = (
    df_league_raw
    .groupby(['SEASON', 'SHOT_ZONE_BASIC'], as_index=False)['FGA']
    .sum()
)

# Compute share of FGA per season
df_league['TOTAL_FGA_SEASON'] = df_league.groupby('SEASON')['FGA'].transform('sum')
df_league['FGA_SHARE'] = df_league['FGA'] / df_league['TOTAL_FGA_SEASON']

df_league.head()


In [ ]:
# Stacked area chart: league-wide shot distribution over time

shot_zone_order = [
    'Restricted Area',
    'In The Paint (Non-RA)',
    'Mid-Range',
    'Left Corner 3',
    'Right Corner 3',
    'Above the Break 3',
    'Backcourt'
]

chart_league = (
    alt.Chart(df_league)
    .mark_area()
    .encode(
        x=alt.X('SEASON:O', title='Season', sort=SEASONS),
        y=alt.Y('FGA_SHARE:Q', stack='normalize', axis=alt.Axis(format='%'), title='Share of FGA'),
        color=alt.Color('SHOT_ZONE_BASIC:N', title='Shot Zone', sort=shot_zone_order),
        tooltip=[
            alt.Tooltip('SEASON:O', title='Season'),
            alt.Tooltip('SHOT_ZONE_BASIC:N', title='Zone'),
            alt.Tooltip('FGA_SHARE:Q', title='Shot Share', format='.1%'),
            alt.Tooltip('FGA:Q', title='FGA (agg)')
        ]
    )
    .properties(
        title='League-wide Shot Distribution by Zone (2000–2025)',
        width=800,
        height=400
    )
)

chart_league


## 2. Event-driven turning points

We define a small table of key rule or strategy changes and annotate them on a timeline that shares the same x-axis as the league-wide chart. You can expand or refine this list for your narrative.


In [ ]:
# Define key events manually
events_data = [
    {'SEASON': '2004-05', 'event': 'Hand-checking rules enforced on perimeter'},
    {'SEASON': '2012-13', 'event': 'Peak "Moreyball" era in Houston (3PT emphasis)'},
    {'SEASON': '2015-16', 'event': 'Stephen Curry unanimous MVP, pull-up 3 revolution'},
    {'SEASON': '2018-19', 'event': 'Freedom of movement rules emphasize spacing'},
]

df_events = pd.DataFrame(events_data)

events_marks = (
    alt.Chart(df_events)
    .mark_rule(color='black', strokeDash=[4,2])
    .encode(
        x=alt.X('SEASON:O', sort=SEASONS),
        tooltip=[alt.Tooltip('SEASON:O', title='Season'), alt.Tooltip('event:N', title='Event')]
    )
)

events_labels = (
    alt.Chart(df_events)
    .mark_text(angle=-90, dy=-10, align='left')
    .encode(
        x=alt.X('SEASON:O', sort=SEASONS),
        text='event:N'
    )
)

timeline_events = (events_marks + events_labels).properties(
    title='Key Rule/Strategy Events',
    width=800,
    height=120
)

timeline_events


## 3. Player-level shot selection evolution

We now fetch shot-zone splits for selected star players (e.g., Stephen Curry, James Harden, LeBron James, Kevin Durant, DeMar DeRozan) and visualize how their shot mixes evolve over time.

> 💡 Tip: To avoid repetitive calls and rate limits, consider caching the `PlayerDashboardByShootingSplits` results per player to CSV/Parquet, then reloading for visualization.


In [ ]:
# Define players of interest by name (we will look up their IDs)
player_names = [
    'Stephen Curry',
    'James Harden',
    'LeBron James',
    'Kevin Durant',
    'DeMar DeRozan'
]

# Resolve players and pick the first match for each name
player_map = {}
for name in player_names:
    matches = find_player_by_name(name)
    if not matches:
        print(f'No NBA API match found for {name}')
        continue
    player_map[name] = matches[0]['id']
    print(f"{name}: {matches[0]['id']}")

player_map


In [ ]:
# Fetch shooting splits for each player across all seasons

player_frames = []

for name, pid in player_map.items():
    print(f'Fetching shooting splits for {name} (ID {pid})...')
    try:
        resp = PlayerDashboardByShootingSplits(
            player_id=pid,
            season_type_playoffs='Regular Season',
            per_mode_detailed='PerGame'
        )
        df = resp.by_zone.get_data_frame()  # 'By Zone' table
        df['PLAYER_NAME'] = name
        player_frames.append(df)
        time.sleep(1.0)
    except Exception as e:
        print(f'Failed to fetch {name}: {e}')

df_players_raw = pd.concat(player_frames, ignore_index=True)
df_players_raw.head()


In [ ]:
# Transform player shooting splits for visualization

# Typical columns: 'SEASON_ID', 'SHOT_ZONE_BASIC', 'FGA', 'FGM', etc.
df_players = df_players_raw.copy()
df_players.rename(columns={'SEASON_ID': 'SEASON'}, inplace=True)

# Compute per-season total FGA for each player
df_players['TOTAL_FGA_PLAYER_SEASON'] = df_players.groupby(['PLAYER_NAME', 'SEASON'])['FGA'].transform('sum')
df_players['FGA_SHARE'] = df_players['FGA'] / df_players['TOTAL_FGA_PLAYER_SEASON']

df_players.head()


In [ ]:
# Player-level shot selection evolution chart (interactive dropdown)

player_dropdown = alt.binding_select(options=sorted(df_players['PLAYER_NAME'].unique()), name='Player: ')
player_select = alt.selection_single(fields=['PLAYER_NAME'], bind=player_dropdown, init={'PLAYER_NAME': 'Stephen Curry'})

chart_player = (
    alt.Chart(df_players)
    .mark_area()
    .encode(
        x=alt.X('SEASON:O', title='Season'),
        y=alt.Y('FGA_SHARE:Q', stack='normalize', axis=alt.Axis(format='%'), title='Share of FGA'),
        color=alt.Color('SHOT_ZONE_BASIC:N', title='Shot Zone', sort=shot_zone_order),
        tooltip=[
            alt.Tooltip('PLAYER_NAME:N', title='Player'),
            alt.Tooltip('SEASON:O', title='Season'),
            alt.Tooltip('SHOT_ZONE_BASIC:N', title='Zone'),
            alt.Tooltip('FGA_SHARE:Q', title='Shot Share', format='.1%'),
            alt.Tooltip('FGA:Q', title='FGA (zone)')
        ]
    )
    .add_selection(player_select)
    .transform_filter(player_select)
    .properties(
        title='Player Shot Selection Evolution by Zone',
        width=800,
        height=400
    )
)

chart_player


## 4. Interactive shot-chart heatmaps over time

We use `ShotChartDetail` to obtain individual shot coordinates (`LOC_X`, `LOC_Y`) for each player and season. Then we create binned 2D heatmaps and add a season slider to show how shot locations migrate over time.

> ⚠️ These API calls can be heavy (one request per season per player). You may want to only pull a subset of seasons, or cache the results.


In [ ]:
def fetch_shotchart_for_player_seasons(player_id: int, seasons: list, season_type='Regular Season'):
    frames = []
    for season in seasons:
        print(f'Fetching ShotChartDetail for player {player_id}, {season}...')
        try:
            resp = ShotChartDetail(
                team_id=0,
                player_id=player_id,
                season_type_all_star=season_type,
                season_nullable=season,
                context_measure_simple='FGA'
            )
            df = resp.get_data_frames()[0]
            df['SEASON'] = season
            frames.append(df)
            time.sleep(1.0)
        except Exception as e:
            print(f'Failed for {season}: {e}')
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)


In [ ]:
# Example: build shotchart data for Stephen Curry over a selected subset of seasons
curry_id = player_map.get('Stephen Curry')

# Choose a narrower range to start, e.g. 2012–2016
curry_seasons = [s for s in SEASONS if s >= '2012-13' and s <= '2015-16']

df_curry_shots = fetch_shotchart_for_player_seasons(curry_id, curry_seasons)
df_curry_shots.head()


In [ ]:
# Build 2D binned heatmap with season slider

if not df_curry_shots.empty:
    season_index_map = {season: i for i, season in enumerate(sorted(curry_seasons))}
    df_curry_shots['season_idx'] = df_curry_shots['SEASON'].map(season_index_map)

    slider = alt.binding_range(min=0, max=len(curry_seasons)-1, step=1, name='Season index: ')
    season_index = alt.selection_single(fields=['season_idx'], bind=slider, init={'season_idx': 0})

    heatmap = (
        alt.Chart(df_curry_shots)
        .mark_rect()
        .encode(
            x=alt.X('LOC_X:Q', bin=alt.Bin(extent=[-250, 250], step=15), title='Court X'),
            y=alt.Y('LOC_Y:Q', bin=alt.Bin(extent=[-50, 420], step=15), title='Court Y'),
            color=alt.Color('count():Q', title='Shot Density'),
            tooltip=[
                alt.Tooltip('SEASON:N', title='Season'),
                alt.Tooltip('count():Q', title='Shots in bin')
            ]
        )
        .add_selection(season_index)
        .transform_filter(season_index)
        .properties(
            title='Stephen Curry Shot Chart Heatmap (Season Slider)',
            width=400,
            height=500
        )
    )

    heatmap
else:
    print('No shot data available for selected seasons.')


## 5. Putting it together for storytelling

In a narrative article (e.g., in a web app or static HTML export), you might lay these out as:

1. **League chart + events timeline**: vertically stacked, sharing the x-axis (season).
2. **Player evolution chart**: below, with explanatory text and specific presets (e.g., start with DeRozan vs Curry).
3. **Shot-chart heatmaps**: used sparingly as focused story moments (e.g., Curry 2012 vs 2016).

You can export each chart as HTML using `chart.save('chart.html')` or integrate the code directly into a web framework (Streamlit, Flask, etc.).
